In [3]:
import scanpy as sc
import rpy2
import os
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr


from rpy2.robjects import pandas2ri
os.chdir("/data")
%load_ext rpy2.ipython






The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [2]:
#adata=sc.read_h5ad("h5ad/batch_corrected.h5ad")

In [3]:
#adata=adata.raw.to_adata()

In [4]:
#adata.obs.annot.isin(["Macrophages", "T Cells", "Epithelial"])

GACGCTGTCGGCTATA-1-cd40-1     True
ATCTCTACAGCGTAGA-1-cd40-1     True
CGGAACCTCCCGAGAC-1-cd40-1     True
GTAACACCAACAGTGG-1-cd40-1     True
TTAGGCAAGATGATTG-1-cd40-1     True
                             ...  
TCCCACATCAATGTCG-1-tak-2     False
GTTAGACTCAGACATC-1-tak-2     False
CAGATACAGCCTCATA-1-tak-2     False
CAGCGTGAGAATTTGG-1-tak-2     False
GAAGAATGTTGGCTAT-1-tak-2     False
Name: annot, Length: 124639, dtype: bool

In [5]:
test=sc.get.aggregate(adata, by=["sample_id", "annot", "condition"], func="sum", mask=adata.obs.annot.isin(["Macrophages", "T Cells", "Epithelial"]))

In [ ]:
test=test.obs[test.annot.isin(["Macrophages", "T Cells", "Epithelial"])]

In [7]:
test_df = pd.DataFrame(test.layers["sum"].T, columns=test.obs.index, index = test.var.index)

In [20]:
%%R -i exp_df,coldata
library(edgeR)
library(dplyr)
coldata$annot <- replace(as.character(coldata$annot), coldata$annot == "T Cells", "T_Cells")
group <- factor(paste(coldata$condition, coldata$annot, sep="."))
coldata <- cbind(coldata, group)
y <- DGEList(exp_df)
y$group <- coldata$group
keep <- filterByExpr(y)
y <- y[keep,keep.lib.sizes=FALSE]
y <- calcNormFactors(y)
design <- model.matrix(~0+group , data=coldata)
colnames(design) <- levels(group)
my.contrasts <- makeContrasts(
    epithelial.takvsctrl = tak.Epithelial - ctrl.Epithelial,
    epithelial.cd40vsctrl = cd40.Epithelial - ctrl.Epithelial,
    epithelial.combovsctrl = combo.Epithelial - ctrl.Epithelial,
    epithelial.takvscd40 = tak.Epithelial - cd40.Epithelial,
    epithelial.combovscd40 = combo.Epithelial - cd40.Epithelial,
    epithelial.combovstak = combo.Epithelial - tak.Epithelial,
    epithelial.takvscd40 = tak.Epithelial - cd40.Epithelial,
    macrophages.takvsctrl = tak.Macrophages - ctrl.Macrophages,
    macrophages.cd40vsctrl = cd40.Macrophages - ctrl.Macrophages,
    macrophages.combovsctrl = combo.Macrophages - ctrl.Macrophages,
    macrophages.takvscd40 = tak.Macrophages - cd40.Macrophages,
    macrophages.combovscd40 = combo.Macrophages - cd40.Macrophages,
    macrophages.combovstak = combo.Macrophages - tak.Macrophages,
    macrophages.takvscd40 = tak.Macrophages - cd40.Macrophages,
    tcells.takvsctrl = tak.T_Cells - ctrl.T_Cells,
    tcells.cd40vsctrl = cd40.T_Cells - ctrl.T_Cells,
    tcells.combovsctrl = combo.T_Cells - ctrl.T_Cells,
    tcells.takvscd40 = tak.T_Cells - cd40.T_Cells,
    tcells.combovscd40 = combo.T_Cells - cd40.T_Cells,
    tcells.combovstak = combo.T_Cells - tak.T_Cells,
    tcells.takvscd40 = tak.T_Cells - cd40.T_Cells, levels=design)
y <- estimateDisp(y, design, robust=TRUE)
fit <- glmQLFit(y, design, robust=TRUE)
qlf <- glmQLFTest(fit, contrast=my.contrasts)
for (i in colnames(my.contrasts)){
    qlf <- glmQLFTest(fit, contrast=my.contrasts[,i])
    test<-topTags(qlf, n=Inf)
    write.csv(test$table, paste0("/data/csv/de/", i, ".csv"))}